# Summary
As a part of my NBA project I've pulled over 100k boxscores by player. This goes over a subset of the work where I needed to determine how many days rest each player had between games.

Pandas and Dataframes are great for manipulating data. As someone coming from Excel/VBA, the power of all the Python libraries is phenomenal (not to mention the massive number of StackOverflow material). Since I am in the process of converting over to Pandas I've been doing my best to try to stay within Python to get immersed in it, which I think is useful exercise.

_Nonetheless there are certain things that are just better in Excel_. If you don't believe me create a custom function in Python that does something weird and try to troubleshoot it--sometimes it's just easier to be able to scroll through a Dataframe cell by cell without having to manually type in filters, etc. Yes you could use widgets or something along those lines to filter the data--but I'd rather use Excel, especially since working within a DataFrame I cut out a lot of the overhead in Excel if I am not using Excel formulas. _Enter the xlwings library!.

## Setup libraries

To get started we need the following libraries:
1. Pandas.
2. Parquet (https://arrow.apache.org/docs/python/parquet.html).
3. xlwings (https://docs.xlwings.org/en/stable/installation.html).
4. Excel.
5. The Python logger.
6. Datetime.

The first step is to load the relevant libraries

In [31]:
import pandas as pd
import xlwings
import logging

## Setup the logger

In [3]:
logging.basicConfig(level=logging.DEBUG, filename='_consol_data.log', filemode='w', format=' %(asctime)s - %(name)s - %(levelname)s - %(message)s')

## The data
Now we have loaded are DataFrame.
1.  'PK' is a key from the original DataFrame. (I use this for the join in my actual use case but we can ignore it here.)
2. 'SEASON' is the NBA season.
3. 'PLAYER_ID' is an individual player (which maps to a name in the larger file).
4. 'GAME_ID' is the ID of the game. In theory this should be sequential but in my file they are not. Take note of this for later.
5. 'GAME_DATE_EST_ADJ' is the date the game took place in Datetime format (this is why I prefer Parquet over CSV files).

In [6]:
df=pd.read_parquet('xl_wings.gzip')
df.shape,df.dtypes

((1013579, 5), PK                           object
 SEASON                       object
 PLAYER_ID                     int64
 GAME_ID                      object
 GAME_DATE_EST_ADJ    datetime64[ns]
 dtype: object)

In [4]:
df.head()

,PK,SEASON,PLAYER_ID,GAME_ID,GAME_DATE_EST_ADJ
index,,,,,
0,0028000001_76011,1980-1981,76011,0028000001,1980-10-10
1,0028000001_78095,1980-1981,78095,0028000001,1980-10-10
2,0028000001_77141,1980-1981,77141,0028000001,1980-10-10
3,0028000001_76436,1980-1981,76436,0028000001,1980-10-10
4,0028000001_77308,1980-1981,77308,0028000001,1980-10-10


## Final setup
Before we perform the calculation we need to make sure the DataFrame is sorted in the right order.

1. Season.
2. Player.
3. Game Date.
4. Game ID. _Again take note of this field._

In [19]:
df.sort_values(by=['SEASON', 'PLAYER_ID','GAME_DATE_EST_ADJ','GAME_ID'],inplace=True)
df.reset_index(inplace=True)

### Days rest calculation*
  - Days rest is a category. The categories are based on the splits used by the NBA. Here is an example, https://stats.nba.com/player/1629029/traditional/ (go to 'Splits').
  - Itertuples returns a tuple containing the row elements. The logic is as follows:
    1. If we are on the first row then just take store the row values (no comparison required), which is the first Else statement.
    2. Calculate the number of days played if the season and player for the current row are the same as for the last row and store in a dictionary.
       - If there are more than 6 days rest categorize as 6+ (as per the NBA methodology).
       - Otherwise tag as is.
    3. Convert the results into a Dataframe and merge back into the original df.
  
 *Note this taken from a larger script that will go up later.

In [20]:
#Rowise comparson
day_diff=None
day_dict={}

for row in df.itertuples():
    if row[0] != 0: 
        if row[3] ==last[3] and row[4] ==last[4]:
            day_diff=(row[6] - last[6]).days
            if day_diff >= 6:
                day_dict[row[2]] = str(min(day_diff,6)) + '+ Day(s) Rest'
            else:
                day_dict[row[2]] = str(day_diff) +' Day(s) Rest'
            day_diff=None
        else:
            day_dict[row[2]]='6+ Day(s) Rest'
        last=row
    else: #first row of the table just store the last value
        day_dict[row[2]]='6+ Day(s) Rest'
        last=row




## Results
For each player-game pair we have a dictionary that categorizes how many day(s) rest there was between games.

Before xlwings, I could only think of three ways to check if this dictionary had the correct values, all of which were tedious:
1. Print the dictionary to the console. _Not feasible with so many key/value pairs._ I'll leave showing 100k rows to your imagination and skip this step.
2. Use the Python logger to create a logfile of the dictionary (or a large subset). Then I could see if there were any odd values.
3. Eyeball the dataframe itself. _Not ideal with so many rows._
4. Export to file, open in Excel and go back-and-forth. _Still tedious._ I'm not going to go over this example since there are losts of good tutorials on exporting from Pandas to Excel.

In [22]:
day_dict['0028000009_229']

'6+ Day(s) Rest'

### Option 2: Logger
This did not look ideal. I could have parsed the file and pulled out what I wanted, but this didn't work. On to the next step.

![Console Log](https://i.imgur.com/7IyOyKi.png)

In [24]:
logging.debug(f'{day_dict}')
logging.shutdown()

### Option 3: Dataframe
I'm comfortable with SQL so my preferred approach was to create a temporary Dataframe and join using the 'PK' field.
1. Create the temporary Dataframe.
2. Merge on the 'PK' field. Since I know the lists should be the same I could do an inner join, however, to be safe I prefer doing a left join to prevent messing with the original. Then if there is an issue I can drop the merged column.

In [26]:
r_days=list(day_dict.items())
df_rest_calc=pd.DataFrame(r_days, columns=['PK','REST_DAYS'])
df_rest_calc.head()

,PK,REST_DAYS
0,0028000009_229,6+ Day(s) Rest
1,0028000011_229,1 Day(s) Rest
2,0028000034_229,4 Day(s) Rest
3,0028000048_229,3 Day(s) Rest
4,0028000060_229,3 Day(s) Rest


In [45]:
df= df.merge(df_rest_calc, left_on='PK', right_on='PK',how='left')
df.head()

,index,PK,SEASON,PLAYER_ID,GAME_ID,GAME_DATE_EST_ADJ,REST_DAYS
0,177,0028000009_229,1980-1981,229,0028000009,1980-10-10,6+ Day(s) Rest
1,209,0028000011_229,1980-1981,229,0028000011,1980-10-11,1 Day(s) Rest
2,682,0028000034_229,1980-1981,229,0028000034,1980-10-15,4 Day(s) Rest
3,957,0028000048_229,1980-1981,229,0028000048,1980-10-18,3 Day(s) Rest
4,1213,0028000060_229,1980-1981,229,0028000060,1980-10-21,3 Day(s) Rest


#### Using the Dataframe
Things look a bit easier to look at with the Dataframe, once we want to go across more than a few rows it becomes a pain (and in my original use case I had columns that went far over to the right).

Now of course this is using relatively clean data (as I've fixed some of the errors). In real life things were a lot more messy:
1. The NBA API is not documented. As a result there is a lot of guessing related to data quality. _First, I need to assume any bad data is on my end_; when something looks off I need to make sure I didn't screw up any transformations. 
2. The assumptions I'm using are just that--assumptions. Since the GAMEID consists of '002' + 'SEASON' + GAME #, I had assumed that the GAMEID could be sorted sequentially. That sounds like it makes sense..._however..._

When I first looked at my rest-day categories thinks looked screwed up. Some of the intervals weren't correct--some were even negative. Since I am new to itertuples, I assume it was a coding error...

Maybe an hour later, after combing through the function and testing with my own samples I couldn't find a bug. **Then I realized that the GAMEIDs are not sequential...so my logic**. This was the issue. Anyway I'll address that in the post about cleaning the NBA data itself.

This made me think there has to be a better way...enter xlwings.

---

## Using XL wings

1. Install the library if you haven't alredy (https://docs.xlwings.org/en/stable/installation.html).
2. I imported it earlier but the convention they use is 'import xlwings as xw' so you use 'xw' to access xlwings.
3. Setup an instance of Excel. 
4. Push your Dataframe to a range in Excel. _Done!._

In [33]:
app = xw.App() #set up Excel instance

In [34]:
xw.Range('A1').value=df.loc[:50000] #load 50,000 rows of the Dataframe to Excel

### Excel output
Now we have the output in Excel and use the Excel GUI to explore our data. Instead of having to export/import the files in Excel everything is there and I can dynamically alter what is pushed by simply changing my selector.
![Excel Output](https://i.imgur.com/8bzVNSC.png)

### Using Pandas and Excel together
For example I can now explore/filter the data in Excel and simply iterate in Pandas.

![Excel Sorting](https://i.imgur.com/jSl5xI2.png) | ![Excel Filters](https://i.imgur.com/lNT5Nke.png)